In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
from tensorflow.keras.models import load_model
import tensorflow_hub as hub
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd

# !wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
import tokenization

2021-12-10 23:43:48.485642: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
/ssoft/spack/arvine/v1/opt/spack/linux-rhel7-skylake_avx512/gcc-8.4.0/python-3.7.7-drpdlwdbo3lmtkcbckq227ypnzno4ek3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: mpi4py.MPI.Status size changed, may indicate binary incompatibility. Expected 40 from C header, got 48 from PyObject
  return f(*args, **kwds)
/ssoft/spack/arvine/v1/opt/spack/linux-rhel7-skylake_avx512/gcc-8.4.0/python-3.7.7-drpdlwdbo3lmtkcbckq227ypnzno4ek3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: mpi4py.MPI.Datatype size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
  return f(*args, **kwds)
/ssoft/spack/arvine/v1/opt/spack/linux-rhel7-skylake_avx512/gcc-8.4.0/python-3.7.7-drpdlwdbo3lmtkcbckq227ypnzno4ek3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: mpi4py.MPI.Request size changed, may indi

In [2]:
MODEL_CHECKPOINT_NAME = "model_lr=5e-06_hid=1024,512,256_maxlen=300_batch=8_epochs=100_seed=72_l2=0.003.h5"
MAX_LEN = 300  # Needs to match the checkpoint above!

In [3]:
# fetch bert layer
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

# initialize tokenizer
tokenizer = tokenization.FullTokenizer(
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy(),
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
)

print(f"BERT layer to be used loaded")

2021-12-10 23:44:11.437125: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-12-10 23:44:11.539281: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-12-10 23:44:11.539351: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
2021-12-10 23:44:11.545318: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-12-10 23:44:11.550174: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2021-12-10 23:44:11.551737: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so

BERT layer to be used loaded


In [ ]:
model = load_model(MODEL_CHECKPOINT_NAME, custom_objects={'KerasLayer': hub.KerasLayer})

print(f"Loaded checkpoint: {MODEL_CHECKPOINT_NAME}")

In [45]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

@dask.delayed
def predict(model, texts_batch):
    batch_input = bert_encode(texts_batch, tokenizer, max_len=MAX_LEN)
    predictions = model.predict(batch_input).round().astype(np.uint8)
    return predictions # np.array([0,1,1,0,0,0,...])

texts_sample = [
    "This is a disaster!",
    "But this is not!",
    "Haiti earthquake 2010!",
    "lava ground!"]
z = predict(model, texts_sample)

print(z)
z.compute()

Delayed('predict-22f4b830-2753-46b5-9cd6-9ef9e25089be')


array([[0],
       [0],
       [1],
       [0]], dtype=uint8)

In [52]:
array = da.from_delayed(z, (4,), dtype=np.uint8)
array

dask.array<from-value, shape=(4,), dtype=uint8, chunksize=(4,), chunktype=numpy.ndarray>

In [53]:
array.compute()

array([[0],
       [0],
       [1],
       [0]], dtype=uint8)

In [66]:
# TODO 1. Make the prediction work on Quotebank
#         Below is my trial
DATASET_PATH = "../quotebank"
quotebank_df_array = [dd.read_parquet(f"{DATASET_PATH}/quotes-{year}.parquet") for year in range(2008, 2021)]
dataset = dd.concat(quotebank_df_array, interleave_partitions=True)

In [72]:
# n = len(dataset)
n = 234994989
n

234994989

In [ ]:
array = da.from_delayed([predict(model, [q]) for q in dataset.quotation.str], (n,), dtype=float)
array

In [ ]:
array.compute()

In [ ]:
# TODO 2. filter quotes belonging to positive predictions
# TODO 3. randomly select 1000 positive quotes. We will pick a subset of them for hand evaluation
# TODO 4. if not too large --> save the resulting quotes dataset somewhere
#         if too large --> save the IDs of the positive quotes
#         or perpahs don't save anything if we will not need it later?